<img src="https://user-images.strikinglycdn.com/res/hrscywv4p/image/upload/c_limit,fl_lossy,h_300,w_300,f_auto,q_auto/1266110/Logo_wzxi0f.png" style="float: left; margin: 20px; height: 55px">


# Advanced SQL - Window Functions

----

### One of the most important things to be aware in a company are sales. We are going to explore the excel_sql_inventory_data and the excel_sql_transaction_data.

#### 1. What’s the product with the highest price.


#### 2. What’s the product with the most inventory.


#### 3. Which product has sold the most in units. (Let’s assume every transaction sells one unit.)

#### 4. Which product has sold the most in sales.


#### 5. Which are the top 5 product type by units sold.


#### 6. If we had sold these product wholesale which would be the top categories.

----

### When working in a company one of the main questions will be around how people is behaving on the site and how users are reaching different steps on the web or app. In order to start looking at this we are going to use the yammer_users table and yammer_events table.


#### 7. Get the number of users that were created per month.


#### 8. Get the number of users that send a message in may 2014. To get people who sent a message filter event_name = ‘send_message’



#### 9. Get the number of users that sent a message within 1 day from being created.


#### 10. I want you to create a table that looks like this.
<img src="Resources/Screenshot 2020-02-28 at 17.54.38.png">

#### 11. I want you to create a table that looks like this.
<img src="Resources/Screenshot 2020-02-28 at 17.54.46.png">


In [ ]:
'''WITH base as(
SELECT
  u.user_id,
  u.created_at,
  e.occurred_at,
  COUNT(e.event_name) OVER (PARTITION BY u.user_id ORDER BY e.occurred_at) AS n_messages
FROM
  tutorial.yammer_users u 
  LEFT JOIN tutorial.yammer_events e 
  ON e.user_id = u.user_id
  and event_name = 'send_message'
  and e.occurred_at < u.created_at + INTERVAL '30 days'
  )
  
SELECT
  DATE_TRUNC('month', created_at) as month,
  COUNT(DISTINCT user_id) as users,
  COUNT(DISTINCT CASE WHEN n_messages > 1 THEN user_id END) as "Send more than 1 messages within 30 days",
  COUNT(DISTINCT CASE WHEN n_messages > 2 THEN user_id END) as "Send more than 2 messages within 30 days",
  COUNT(DISTINCT CASE WHEN n_messages > 3 THEN user_id END) as "Send more than 3 messages within 30 days",
  COUNT(DISTINCT CASE WHEN n_messages > 5 THEN user_id END) as "Send more than 5 messages within 30 days"
FROM
  base
GROUP BY
  1
HAVING
  COUNT(DISTINCT CASE WHEN n_messages > 1 THEN user_id END) > 0
ORDER BY
  1 
'''

#### 12. What’s the most popular event performed after signup. What do people tend to do after signup by month? Note: Get only the most popular event after signup by month and the number of users do do it exactly after signup. Not at any point.
<img src="Resources/Screenshot 2020-02-28 at 17.54.53.png">

In [ ]:
'''WITH events_post_signup as(
SELECT
  user_id,
  occurred_at,
  event_name,
  COUNT(event_name) OVER (PARTITION BY user_id ORDER BY occurred_at ASC) AS events
FROM
  tutorial.yammer_events
ORDER BY
  1,2
),

most_pop_event as(
SELECT
  DATE_TRUNC('month', occurred_at) as month,
  event_name,
  count(DISTINCT user_id) as users,
  RANK() OVER (PARTITION BY DATE_TRUNC('month', occurred_at) ORDER BY count(DISTINCT user_id) DESC) as rank
FROM
  events_post_signup
WHERE
  events = 2
GROUP BY
  1,2
ORDER BY
  1,3 DESC
)

SELECT
  month,
  event_name,
  users
FROM
  most_pop_event
WHERE
  rank = 1
'''

#### 13. What are the most popular events on the second session of a user(Assume a new session is created after 10 minutes of inactivity? Get the 5 most popular events on the second session of a user by month.
<img src="Resources/Screenshot 2020-02-28 at 17.54.58.png">


In [ ]:
'''WITH events_after_inact as(
SELECT
  sub.*,
  RANK () OVER (PARTITION BY sub.user_id ORDER BY sub.occurred_at)
FROM (
      SELECT
        user_id,
        event_name,
        occurred_at,
        occurred_at - LAG(occurred_at, 1) OVER (PARTITION BY user_id ORDER BY occurred_at) as difference
      FROM
        tutorial.yammer_events
      WHERE
        event_name != 'login'
      ) sub
WHERE sub.difference >= '00:10:00'
),

second_session_event as (
SELECT
  user_id,
  event_name,
  occurred_at
FROM
  events_after_inact
WHERE
  rank = 1
),

top_5_events as (
SELECT
  DATE_TRUNC('month', occurred_at) as month,
  event_name,
  COUNT(DISTINCT user_id) as users,
  RANK() OVER (PARTITION BY DATE_TRUNC('month', occurred_at) ORDER BY count(DISTINCT user_id) DESC) as rank
FROM
  second_session_event
GROUP BY
  1,2
ORDER BY
  1,4
)

SELECT
  month,
  event_name,
  users,
  RANK() OVER (PARTITION BY month ORDER BY users DESC) as rank
FROM
  top_5_events
WHERE
  rank < 6
'''